### Cloning Repo and install requirements

In [ ]:
import os
!git clone https://github.com/JiayuanWang-JW/YOLOv8-multi-task
os.chdir('/content/YOLOv8-multi-task')
!pip install --quiet -e .
os.chdir('/content')
!pip install --quiet ultralytics
# !pip uninstall -y albumentations ## Data augmentations did not work with multitask
!rm -r sample_data
os.kill(os.getpid(), 9) # Restart Runtime

Cloning into 'YOLOv8-multi-task'...
remote: Enumerating objects: 1698, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (234/234), done.
remote: Total 1698 (delta 105), reused 196 (delta 49), pack-reused 1408
Receiving objects: 100% (1698/1698), 19.55 MiB | 11.13 MiB/s, done.
Resolving deltas: 100% (201/201), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 401.1 kB/s eta 0:00:00


$$$$

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## download Data

In [ ]:
!gdown 1iBpVko2sehPeOPhQXWaujOpKFQmNvd_m
!unrar x -y -idq /content/RailSem19.rar RailSem19
!rm /content/RailSem19.rar

Downloading...
From (original): https://drive.google.com/uc?id=1iBpVko2sehPeOPhQXWaujOpKFQmNvd_m
From (redirected): https://drive.google.com/uc?id=1iBpVko2sehPeOPhQXWaujOpKFQmNvd_m&confirm=t&uuid=4af8cbb9-7e3a-467b-8dd0-033b124b2dd7
To: /content/RailSem19.rar
100% 5.41G/5.41G [00:48<00:00, 110MB/s] 


$$$$

## Initialise Model

In [ ]:
import os
import torch
from ultralytics import YOLO
import torch.nn as nn
import ultralytics
import sys

In [ ]:
model = YOLO('/content/drive/MyDrive/Stage 2024/best 3.pt')

$$$$

## Predict Video

**Warmup Model**

In [ ]:
# Make prediction on random image to prepare Predictor and AutoBackend Model for Inference
res = model.predict(source=np.random.randint(0,256,
                                             size=(720,1280,3),
                                             dtype=np.uint8),
                    imgsz=(640,640),
                    name='Rail',
                    conf=0.2,
                    iou=0.25,
                    speed = True)


0: 384x640 (no detections), 110.0ms


In [ ]:
from ultralytics.yolo.data import load_inference_source

In [ ]:
import numpy as np
import cv2
from tqdm import tqdm
from PIL import Image

**Helper function**

In [ ]:
def read_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}")
        return

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Video Path: {video_path}")
    print(f"Frame Count: {frame_count}")
    print(f"Frame Rate: {frame_rate} fps")
    print(f"Frame Dimensions: {width}x{height}")

    # Close the video file
    cap.release()
    return frame_count,frame_rate

**Prepare Dataset from the video**

In [ ]:
source = "ligne 14 cropped.MP4" # Path to video
dataset = load_inference_source(source=source,
                                imgsz=model.predictor.imgsz,
                                vid_stride=model.predictor.args.vid_stride)
frame_count,frame_rate = read_video(source)

**Prediction Parameters**

In [ ]:
color_map = {
    1: (152, 251, 152), # Track
    2: (255, 255, 255),  # Rail
    3: (0,0,255) # Pole
}

alpha = 0.2
model.predictor.args.conf = 0.2
model.predictor.args.iou = 0.2

**Predict Frames**

In [ ]:
frames = []
for batch in tqdm(dataset,total=frame_count):

  path, im0s, vid_cap, s = batch
  im0s = cv2.cvtColor(im0s[0], cv2.COLOR_RGB2BGR)
  im0s = [cv2.resize(im0s, (1280, 720))]
  im = model.predictor.preprocess(im0s)
  preds = model.predictor.model(im)

  results = []
  for i, pred in enumerate(preds):
      if isinstance(pred, tuple):
          pred = model.predictor.postprocess_det(pred, im, im0s)
          results.append(pred)
      else:
          pred = model.predictor.postprocess_seg(pred)
          results.append(pred)

  # extract mask from preds
  mask = np.zeros(shape=(im0s[0].shape[:-1]),dtype=np.uint8)
  for i in range(len(results)-1):
    pred_mask = results[i+1].squeeze().cpu().numpy().astype(np.uint8)
    mask[pred_mask == 1] = i+1


  rgb_mask = np.zeros(im0s[0].shape, dtype=np.uint8)

  for value, color in color_map.items():
      rgb_mask[mask == value] = color
  # relay mask on the original image
  blended_image = np.array(Image.blend(Image.fromarray(im0s[0]), Image.fromarray(rgb_mask), alpha))
  results[0][0].orig_img = blended_image # cv2.cvtColor(blended_image, cv2.COLOR_RGB2BGR)
  frames.append(results[0][0].plot()[...,::-1])

**Save Predicted Video**

In [ ]:
out = cv2.VideoWriter(f"pred_{source}",cv2.VideoWriter_fourcc(*'DIVX'), frame_rate, (1280, 720))

for i in tqdm(range(len(frames))):
    out.write(frames[i])
out.release()
print("Video saved successfully !!!")